In [1]:
import os
import requests
import sys
import time

In [14]:
# AQI
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [7]:
def get_html():
    for year in range(2013,2019):
        for month in range(1,13):
            if month < 10:
                url = 'https://en.tutiempo.net/climate/0{}-{}/ws-432950.html'.format(month,year)
            else:
                url = 'https://en.tutiempo.net/climate/{}-{}/ws-432950.html'.format(month,year)
        
            page = requests.get(url)
            page_utf = page.text.encode('utf=8') #contains entire html
        
            if not os.path.exists('DataP/html_data/{}'.format(year)):
                os.makedirs('DataP/html_data/{}'.format(year))
        
            with open('DataP/html_data/{}/{}.html'.format(year,month), 'wb') as output:
                output.write(page_utf)
        sys.stdout.flush()

In [8]:
start = time.time()
get_html()
stop = time.time()
print('Time taken {}'.format(stop-start))

Time taken 176.40519428253174


In [21]:
# AQI
def get_avg():
    aqi_df = pd.DataFrame()
    for year in range(2013,2019):
        avg_lst = []
        for rows in pd.read_csv('Data/AQI/aqi{}.csv'.format(year), chunksize=24):
            df = pd.DataFrame(data=rows)
            sum = 0
            avg = 0.0
            pm2 = df['PM2.5']
            for i in pm2:
                if type(i) is int or type(i) is float:
                    sum = sum+i
                elif type(i) is str:
                    if i!='NoData' and i!='PwrFail' and i!='---' and i!='InVld':
                        temp = float(i)
                        sum = sum+temp
            avg = sum/24
            avg_lst.append(avg)
        aqi_df['{}'.format(year)] = pd.Series(avg_lst)
    return aqi_df

In [22]:
start = time.time()
df = get_avg()
stop = time.time()
print('Time taken {}'.format(stop-start))

Time taken 19.39517378807068


In [23]:
df.head()

,2013,2014,2015,2016,2017,2018
0,284.795833,255.875000,172.750000,284.795833,284.795833,255.875000
1,219.720833,217.083333,146.791667,219.720833,219.720833,217.083333
2,182.187500,229.166667,113.875000,182.187500,182.187500,229.166667
3,115.287500,392.208333,144.750000,115.287500,115.287500,392.208333
4,0.000000,536.500000,118.458333,0.000000,0.000000,536.500000
